In [1]:
import gc
import math
import os
import pathlib
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm_notebook

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split

import lightgbm as lgb
import xgboost as xgb

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [4]:
def features_maker(df, first_index=None, last_index=None, smootch_windows_size = (3, 5, 7)):
    if first_index == None or last_index == None:
        first_index = 0
        last_index = df.shape[0] - 1

    smooth_feature_names = ['smooth_feature_{}_ws_{}'.format(i, window_size) for i, window_size in enumerate(smootch_windows_size)]
    for current_index in df.indexes.tolist():
        for smooth_feature_name, window_size in zip(smooth_feature_names, smootch_windows_size):
            half_window_size = window_size % 2
            data_series = df['acoustic_data']
            if current_index < first_index + half_window_size:
                smooth_feature_value = data_series.iloc[first_index:first_index + window_size].mean()
            elif current_index < last_index - half_window_size:
                smooth_feature_value = data_series.iloc[last_index - window_size:last_index].mean()
            else:
                smooth_feature_value = data_series.iloc[i - half_window_size:i + half_window_size].mean()
            df.iloc[i][feature_name] = data_series[i] - smooth_feature_value
    return df

In [5]:
earthquake_margin_indexes =[
    5656573,
    50085877,
    104677355,
    138772452,
    187641819,
    218652629,
    245829584,
    307838916,
    338276286,
    375377847,
    419368879,
    461811622,
    495800224,
    528777114,
    585568143,
    621985672
]

In [8]:
earthquakes_length = [earthquake_margin_indexes[i + 1] - earthquake_margin_indexes[i] for i in range(len(earthquake_margin_indexes) - 1)]

In [9]:
earthquakes_length

[44429304,
 54591478,
 34095097,
 48869367,
 31010810,
 27176955,
 62009332,
 30437370,
 37101561,
 43991032,
 42442743,
 33988602,
 32976890,
 56791029,
 36417529]

In [10]:
621985672 - 585568143

36417529

In [11]:
50085877 - 5656573

44429304

In [16]:
complete_earthquaces_length = earthquakes_length[:-1]

In [15]:
#complete_earthquaces_length = complete_earthquaces_length[:-1]

In [17]:
complete_earthquaces_length

[44429304,
 54591478,
 34095097,
 48869367,
 31010810,
 27176955,
 62009332,
 30437370,
 37101561,
 43991032,
 42442743,
 33988602,
 32976890,
 56791029]

In [18]:
len(complete_earthquaces_length)

14

In [25]:
earthquages_length_more_2_less_3 = [length for length in complete_earthquaces_length if length > 20000000 and length < 30000000]

In [20]:
earthquages_length_less_2

[27176955]

In [21]:
earthquages_length_more_3_less_4 = [length for length in complete_earthquaces_length if length > 30000000 and length < 40000000]

In [22]:
earthquages_length_more_4_less_5 = [length for length in complete_earthquaces_length if length > 40000000 and length < 50000000]

In [23]:
earthquages_length_more_5_less_6 = [length for length in complete_earthquaces_length if length > 50000000 and length < 60000000]

In [24]:
earthquages_length_more_6 = [length for length in complete_earthquaces_length if length > 60000000]

In [26]:
earthquages_length_more_3_less_4

[34095097, 31010810, 30437370, 37101561, 33988602, 32976890]

In [27]:
earthquages_length_more_2_less_3

[27176955]

In [28]:
earthquages_length_more_3_less_4

[34095097, 31010810, 30437370, 37101561, 33988602, 32976890]

In [29]:
earthquages_length_more_4_less_5

[44429304, 48869367, 43991032, 42442743]

In [30]:
earthquages_length_more_5_less_6

[54591478, 56791029]

In [31]:
earthquages_length_more_6

[62009332]